In [1]:
### IMPORTATION
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import math
import sys

# Depolimerisation - Model error

## Summary

Step 1 : Discretize the parabolic hyperbolic example

Step 2 : Study the well posedness of the Cauchy problem 

Step 3 : Find properties of the strongly continuous semi-group (as Gâteaux differential ?)

Step 4 : Apply the theorem of first order optimal condition 

Step 5 : Find associated Riccati equations

In [2]:
### PHYSICAL PARAMETERS
L = 10.0  # Domain size
T = 20.0  # Integration time

### NUMERICAL PARAMETERS
NT = 100  # Number of time steps
NX = 100   # Initial number of grid points

In [3]:
### OBSERVER - MOMENT OPERATOR
# First moment
operator_moment_1= L/NX*np.linspace(L/NX, L, NX)
# Second moment
operator_moment_2= L/NX*np.square(np.linspace(L/NX, L, NX))
# Concatenate
observer = np.vstack((operator_moment_1, operator_moment_2))

## Direct computation

#### Lifshitz-Slyosov
Convection-diffusion equation

$
\begin{cases}
 \displaystyle \frac{\partial y}{\partial t}
 - b \frac{\partial} {\partial x}(y +\epsilon \frac{\partial}{\partial x}y) = 0 
 & (x,t) \in (0,\ell) \times (0,T)  \\[0.1cm]
 y(x,0) = y_{0} (x)  \\
 y(t,0) = r(t) \\
 y(L,t) = 0
\end{cases}
$

#### Discretization

Centered

$ \displaystyle \partial_x = \frac{\partial}{\partial x} =  \frac{y_{i+1}-y_{i-1}}{2h}$

$ \displaystyle \partial_xx = \frac{\partial^2}{\partial x^2} =  \frac{y_{i+1}-2y_i+y_{i-1}}{2h} $

Implicit

$ 
\begin{cases} 
\displaystyle \frac{ y^{n+1}_i - y^n_i }{\delta t} - b \frac{ y^{n+1}_{i+1} - y^{n+1}_{i-1} }{2h} - b \epsilon \frac{ y^{n+1}_{i+1} - 2y^{n+1}_{i} + y^{n+1}_{i-1} }{2h} = 0 & \forall i \in [2,N-1] \\[0.1cm]
\displaystyle \frac{ y^{n+1}_0 - y^n_0 }{\delta t} - b \frac{ y^{n+1}_{1}-y^{n+1}_{0}}{h} 
- b \epsilon \frac{ y^{n+1}_{1} - 2y^{n+1}_{0} }{2h} = 0\\
\displaystyle \frac{ y^{n+1}_{N} - y^n_{N} }{\delta t} - b \frac{ - y^{n+1}_{N-1} }{h} - b \epsilon \frac{ - 2y^{n+1}_{N} + y^{n+1}_{N-1} }{h} = 0
\end{cases}
$

and

$ 
\begin{cases} 
cfl = b \frac{\delta t}{2h}\\
y^n_i = y^{n+1}_i - cfl (y^{n+1}_{i+1} - y^{n+1}_{i-1})- cfl \frac{\epsilon}{h} (y^{n+1}_{i+1} - 2y^{n+1}_{i} + y^{n+1}_{i-1}) & \forall i \in [2,N-1] \\[0.1cm]
y^n_0 = y^{n+1}_0 - cfl (2y^{n+1}_{1}-2y^{n+1}_{0})- cfl \frac{\epsilon}{h} (y^{n+1}_{1} - 2y^{n+1}_{0})\\
y^n_N = y^{n+1}_N - cfl (- y^{n+1}_{N-1})- cfl \frac{\epsilon}{h} (- 2y^{n+1}_{N} + y^{n+1}_{N-1})= 0
\end{cases}
$


Computation of flow matrix

$
Y^n =  \left( \begin{array}{c}
    y^n_0\\
    \vdots \\
	y^n_i\\
	 \vdots \\
    y^n_N
	 \end{array} \right) = F(b,\epsilon)^{-1} Y^{n+1}
$


$
\begin{cases}
 \displaystyle cfl = b \frac{\delta t}{2h}\\
 F(b,\epsilon)^{-1} = I_N - cfl( T + \frac{\epsilon}{h} D)
 \end{cases}
$

with

$
 T = \left( \begin{array}{cccc}
	-2      &    2    &   0    & 0 \\
	-1     &     0   & \ddots & 0 \\
	0      & \ddots  & \ddots & 1 \\
	0      & 0       &   -1   & 0
	 \end{array} \right).
$

and

$     
D = \left( \begin{array}{cccc}
	-2     &    1    &   0    & 0 \\
	 1     &    -2   & \ddots & 0 \\
	0      & \ddots  & \ddots & 1 \\
	0      & 0       &    1   & -2
	 \end{array} \right).
$

In [4]:
## DATA GENERATION ############################################################################

### INITIAL CONDITION
x = np.linspace(0.0,L,NX)
sigma = L/10
gaussienne = L/10*np.exp(-(x-L/2)**2/(2*sigma**2)) # Gaussienne
gaussienne[NX-1]=0

### VECTOR INITIALISATION
state = np.zeros((NX,NT)) # Saving the solution of the transport equation
state[:,0] = gaussienne.copy()
mu = np.zeros((2,NT)) # First and second order moment
mu[:,0] = np.dot(observer,state[:,0])

### DEPOLIMERISATION PARAMETER b
b = 0.5  # Depolymerisation speed

### DIFFUSION PARAMETER e
e = 1.0 # Diffusion Speed


### DYNAMIC FLOW PHI
def Flow(l,nx,t,nt,c,eps):
    # CFL condition
    CFL = c*(t/nt)*(nx/l)/2
    # Transport term
    trsp = CFL*(np.diag(np.ones(NX-1),1)\
    -np.diag(np.ones(NX-1),-1))
    trsp[0,0] = -CFL*2
    trsp[0,1] = CFL*2
    
    # Diffusion term
    diffu = eps*CFL*(NX/L)/2*(-2*np.eye(NX,NX)\
    +np.diag(np.ones(NX-1),1)\
    +np.diag(np.ones(NX-1),-1))
    # Final flow computatiom
    flow_i = np.eye(NX,NX) - trsp - diffu
    flow_f = np.linalg.inv(flow_i)
    return flow_f

flow = Flow(L,NX,T,NT,b,e)


In [5]:
### DIRECT COMPUTATION OF THE SOLUTION
for i in range(0,NT-1):
    # Dynamic computation
    state[:,i+1] = np.dot(flow,state[:,i]) 
    # Calculation of the moment
    mu[:,i+1] = np.dot(observer,state[:,i+1])

In [6]:
### PLOTTING SOLUTION OF 1D TRANSPORT EQUATION
# Set up the figure, the axis, and the plot element we want to animate
fig_state = plt.figure()
ax_state = plt.axes(xlim=(0, L), ylim=(0, L/10+1))
line_state, = ax_state.plot([], [], lw=2)

# Initialization function: plot the background of each frame
def Initstate():
    line_state.set_data([], [])
    return line_state,

# Animation function.  This is called sequentially
def Animate1DT(i):
    x = np.linspace(0, L, NX)
    y = state[:,i]
    line_state.set_data(x, y)
    return line_state,

anims = animation.FuncAnimation(fig_state, Animate1DT, \
init_func=Initstate,\
frames=NT, interval=20, blit=True)

# create the HTLM figure
plt.close(anims._fig)
HTML(anims.to_html5_video())

# equivalent to rcParams['animation.html'] = 'html5'
rc('animation', html='html5')

anims



## Well-posedness of the Cauchy problem

#### Continuous
cf. 2.7.1 page 99 of Control and Nonlinearity - Jean-Michel Coron

$
\begin{cases}
 \displaystyle \frac{\partial y}{\partial t}
 - b \frac{\partial} {\partial x}(y +\epsilon \frac{\partial}{\partial x}y) = 0 
 & (x,t) \in (0,\ell) \times (0,T)  \\[0.1cm]
 y(x,0) = y_{0} (x)  \\
 y(t,0) = r(t) \\
 y(L,t) = 0
\end{cases}
$

With $A = - b \frac{\partial} {\partial x}(y +\epsilon \frac{\partial}{\partial x}y)$ and $r(t)=0$, there exists a strongly continuous semi-group of continuous linear operator on $H^{-1}(0,L)$  denoted $S(t)$ and that the infinitesimal generator of this semi-group is $A$.

The the unique solution of the Cauchy problem is a function $y \in C^0((0,T), H^{-1})$ such that :

$ \forall z^{\tau} \in H^{-1} \quad (y(\tau),z^{\tau})_{H^{-1}} - (y(0),S(\tau)^*z^{\tau})_{H^{-1}} =0$


We would like to apply a theorem like ....

Theorem :
Soient $E$ un espace vectoriel normé, $U$ un ouvert de $E$ et $\mathscr{J} : U$ → $R$ une fonctionnelle. Si $x_0 \in U$ est tel que
$\forall x \in U, \mathscr{J}(x_0) \leq \mathscr{J}(x)$, et si $\mathscr{J}$ est Gateaux-différentiable en $x_0$, alors :

$\mathrm{d}_G\mathscr{J}(x_0) = 0$



## Kalman Filter

#### Discrete Functional
Functional $\mathscr{J}$ (discrete) for time $T = Nt \times \delta t $.

$
t_j = j \times \delta t \quad \forall j \in (0,Nt)
$

$
\mathscr{J}(Nt) = \mathscr{J}(Nt)(\zeta,b,\epsilon) = \alpha \parallel \zeta \parallel _{N_{\diamond},\mathscr{Y}}^2 + \beta b^2 + \mu \epsilon^2 + \gamma \sum_{j=0}^{Nt} \delta t \parallel z(t_j) - CF(b,\epsilon)^j(Y_{\diamond}+\zeta) \parallel_{M_j,\mathscr{Z}}^2
$ 

$
\begin{cases}
 \displaystyle cfl = b \frac{\delta t}{2h}\\
 F(b,\epsilon)^{-1} = I_N - cfl( T + \frac{\epsilon}{h} D) \\
 Y^{Nt+1} = Y^{Nt} \\
 Y^{0} = Y_{\diamond} + \zeta
 \end{cases}
$

We are searching $\zeta_{|Nt}$ such that its verify :

$ (\zeta_{|Nt}, b_{|Nt}, \epsilon _{|Nt}) = argmin \mathscr{J}(Nt)$

Since $\mathscr{J}(Nt)$ is two times differentiable in $(\zeta, b, \epsilon) $, them we have an optimal necessary condition on an open space $\mathbb{R}^{N+2}$ that :

$
\mathrm{d} \mathscr{J}(Nt)(\zeta_{|Nt}, b_{|Nt}, \epsilon _{|Nt}) = 0
$

and in particular we have the same equality than the linear case :

$
\partial_{\zeta} \mathscr{J}(Nt)(\zeta_{|Nt}, b_{|Nt}, \epsilon _{|Nt}) = 0
$

This condition became sufficient when there exist a positiv real $\alpha$ such that  :

$ \forall h \in \mathbb{R}^{N+2} \quad \mathrm{d} ^2 \mathscr{J}(Nt)(h,h) \geq \alpha \parallel h \parallel ^2 $

#### Riccati Equation

We have no a priori on the solution so we assume that $Y_{\diamond} = 0$ :

$
	\begin{cases}
		\text{Initialisation :}\\
		P_0^- = P_{\diamond} = \mathbb{1} \\
        P_{trsp} = \beta^-1 (F-I_N) \\
        P_{diffu} =  \mu^-1  D \\
		Y_0^- = Y_{\diamond} = 0\\
        \zeta_0^- = \zeta_{\diamond}\\
        b_0^- = b_{\diamond} \\
        \epsilon_0^- = \epsilon_{\diamond}\\
		\text{Correction :}\\
		K_n = P_n^- C^* (\tilde{M}_n^{-1} + C^*P_n^-C)^{-1} \\
		P_n^+ = (\mathbb{I} - K_nC)P_n^-(\mathbb{I}-K_nC)^* + K_n \tilde{M}_n^{-1} K_n^*\\
		Y_n^+ = Y_n^- + P_n^+C^* \tilde{M}_n ( Z_n - CY_n^-)\\
        \zeta_n^+ = \zeta_n^- + P_n^+C^* \tilde{M}_n ( Z_n - CY_n^-)\\
        b_{n}^+ = b_{n}^- + \frac{n+1}{b_{n}^-}\zeta_{n}^- P_{trsp} C^* \tilde{M}_n ( Z_n - CY_n^-)\\
        \epsilon_{n}^+ = \epsilon_{n}^- + (n+1) b_{n}^- \zeta_{n}^- P_{diffu}C^* \tilde{M}_n (Z_n - CY_n^-)\\
		\text{Prediction :}\\
		P_{n+1}^- = F(b,\epsilon) P_n^+ F(b,\epsilon)^*\\
        P_{trsp} = FP_{trsp}\\
        P_{diffu} = FP_{diffu}\\
		Y_{n+1}^- = F(b,\epsilon) Y_n^+\\
        \zeta_{n+1}^- = \zeta_{n+1}^+ \\
        b_{n+1}^+ = b_{n}^-\\
        \epsilon_{n+1}^- = \epsilon_{n+1}^+ 
	\end{cases}	
$

Demonstration :

$
\begin{split}
\partial _{\zeta} \mathscr{J}(Nt) &= 2\alpha N_{\diamond} \zeta + 2\gamma \delta t \sum_{j=0}^{Nt} F(b, \epsilon)^{j*} C^* M_j(z(t_j)-CY(t_j)) \\
                           & = 2 (\alpha N_{\diamond} + \gamma \delta t \sum_{j=0}^{Nt} F(b, \epsilon)^{j*}C^*M_j  C F(b, \epsilon)^j) \zeta + 2\gamma \delta t \sum_{j=0}^{Nt} F(b, \epsilon)^{j*} C^* M_j(z(t_j))
\end{split}
$ 

where $*$ is the transpose operator for matrix.
Let's define for time $n \in (0,Nt)$ the covariance matrix :

$
P_n = (\alpha N_{\diamond} + \gamma \delta t \sum_{j=0}^{Nt} F^{j*}C^*M_jCF^j)^{-1}
$

Then we have :

$
\partial_{\zeta} \mathscr{J}(Nt)(\zeta_{|Nt}, b_{|Nt}, \epsilon _{|Nt}) = 0
$

$\implies$

$\zeta_{Nt} = P_{Nt} (\gamma \delta t \sum_{j=0}^{Nt} F(b,\epsilon)^{j*} C^* M_j(z(t_j))$

The problem is then reduce to the previous linear case where the parameters are already know. We apply the matrix inverdsion lemma (Woodbury identity) and find a relation between $y_{|n+1}$ and $y_{|n}$.

Now we would like to study the other derivative of the functional.

$
\begin{split}
\partial_{b} \mathscr{J}(Nt) &= 2 \beta b - 2 \gamma \delta t \sum_{j=1}^{Nt} j[z(t_j) ^*M_j CF^{j-1} \partial_b F \zeta - \zeta^* F^{j*} C^*M_j CF^{j-1} \partial_b F  \zeta]  \\
                             & = 2 \beta b - 2 \gamma \delta t \sum_{j=1}^{Nt} j[z(t_j) ^*M_j C F^{j-1} (T+ \epsilon D) \zeta - \zeta^* F^{j*} C^*M_jCF^{j-1} (T+ \epsilon D)  \zeta]
\end{split}
$

$
\partial_{b} \mathscr{J}(Nt)(\zeta_{|Nt}, b_{|Nt}, \epsilon _{|Nt}) = 0
$

$\implies$

$
\begin{split}
b_{|Nt} &= \gamma / \beta \delta t \sum_{j=1}^{Nt} j[z(t_j)^* M_j C F^{j-1} (T+ \epsilon D) \zeta - \zeta^* F^{j*} C^* M_j C F^{j-1} (T+ \epsilon D)  \zeta] \\
              & = b_{Nt-1} + \gamma /\beta \delta t Nt [z(t_{Nt})^* M_{Nt} C F^{Nt-1} (T+ \epsilon D) \zeta - \zeta^* F^{Nt*} C^* M_{Nt} C F^{Nt-1} (T+ \epsilon D)  \zeta] \\
              & = b_{Nt-1} + Nt / \beta [F^{Nt-1}(T+\epsilon D) \zeta ]^* C^* \tilde{M}_{Nt}(z(t_{Nt}) - C F^{Nt} \zeta) \\
              & = b_{Nt-1} + Nt / (b\beta) [F^{Nt-1}(F-I_N) \zeta ]^* C^* \tilde{M}_{Nt}(z(t_{Nt}) - C F^{Nt} \zeta) \\
              & = b_{Nt-1} + Nt/b \zeta^* P_{trsp,n}C^* \tilde{M}_{Nt}(z(t_{Nt}) - C F^{Nt} \zeta)
\end{split}
$

And we use the same demonstration to obtain :

$
\begin{split}
\epsilon_{|Nt} &= \epsilon_{Nt-1} + Nt / \mu [F^{Nt-1}(bD) \zeta ]^* C^* \tilde{M}_{Nt}(z(t_{Nt}) - C F^{Nt} \zeta) \\
             & = \epsilon_{Nt-1} + Nt b \zeta^* P_{diffu,n}C^* \tilde{M}_{Nt}(z(t_{Nt}) - C F^{Nt} \zeta)
\end{split}
$


In [7]:
### KALMAN FILTER #################################################################################

### INITIALISATION
# Construction of the observer (equal zero for the initial condition taken as parameter)
observer_inter = np.vstack((operator_moment_1, np.zeros(NX), \
operator_moment_2, np.zeros(NX)))\
.reshape(2,2*NX)
observer_kalman = np.concatenate((observer_inter, np.zeros((2,2))),axis=1)

# Construction of the norm of the two spaces
standart_deviation = 0.0001
norm_observation = (T/NT)*(1/standart_deviation)**2*np.eye(2)
inv_norm_observation = (NT/T)*(standart_deviation)**2*np.eye(2)

# Initialisation of the covariance matrix
covariance_operator_m = np.kron(np.ones((2,2)),np.eye(NX))
covariance_operator_m = np.concatenate((covariance_operator_m,np.ones((2,2*NX))))
covariance_operator_m = np.concatenate((covariance_operator_m,np.ones((2*NX+2,2))),axis=1)
covariance_operator_p = covariance_operator_m.copy()


# Initialisation of the state
state_init = (sigma*np.sqrt(2*np.pi))**-1*np.exp(-(x-L/3)**2/(2*sigma**2))
state_init[NX-1] = 0
state_m = np.concatenate((\
np.kron(np.ones(2),state_init),10*np.ones((2))))

state_p = state_m.copy()
state_kalman = np.zeros((2*NX+2,NT))



In [8]:
### KALMAN FILTER
for k in range(0,NT):
    # Saving the solution
    state_kalman[:,k] = state_m.copy()

    ### CORRECTION
    # Covariance computation +
    interim_matrix = inv_norm_observation \
    + observer_kalman.dot(covariance_operator_m).dot(observer_kalman.transpose())
    kalman_gain = covariance_operator_m\
    .dot(observer_kalman.transpose())\
    .dot(np.linalg.inv(interim_matrix))

    covariance_operator_p = (np.eye(2*NX+2) \
    - kalman_gain.dot(observer_kalman)).dot(covariance_operator_m)\
    .dot(np.transpose((np.eye(2*NX+2) - kalman_gain.dot(observer_kalman)))) \
    + kalman_gain.dot(inv_norm_observation).dot(np.transpose(kalman_gain))

    # State correction computation +
    state_p = state_m + covariance_operator_p.dot(observer_kalman.transpose()).dot(norm_observation)\
    .dot(mu[:,k]- np.dot(observer_kalman,state_m))

    ### PREDICTION
    # Computation of the flow linked to the new parameters (b,e)
    flow_be = Flow(L,NX,T,NT,state_p[2*NX], state_p[2*NX+1])
    # Construction of the new dynamic (identity for the initial condition)
    flow_kalman = np.concatenate((np.concatenate((flow,np.zeros((NX+2,NX)))),\
    np.concatenate((np.zeros((NX,NX+2)),np.eye(NX+2)))),axis=1)

    # Covariance computation -
    covariance_operator_m = flow_kalman.dot(covariance_operator_p).dot(flow_kalman.transpose())

    # State prediction computation -
    state_m = np.dot(flow_kalman,state_p)



In [9]:
### PLOTTING KALMAN
# Set up the figure, the axis, and the plot element we want to animate
fig_kalman = plt.figure()
ax_kalman = plt.axes(xlim=(0, L), ylim=(0, L/10+1))
line_kalman, = ax_kalman.plot([], [], lw=2)

# Initialization function: plot the background of each frame
def Initkalman():
    line_kalman.set_data([], [])
    return line_kalman,

# Animation function.  This is called sequentially
def Animatek(i):
    x = np.linspace(0, L, NX)
    y = state_kalman[NX:2*NX,i]
    line_kalman.set_data(x, y)
    return line_kalman,

animk = animation.FuncAnimation(fig_kalman, Animatek, \
init_func=Initkalman,\
frames=NT, interval=20, blit=True)

# create the HTLM figure
plt.close(animk._fig)
HTML(animk.to_html5_video())

# equivalent to rcParams['animation.html'] = 'html5'
rc('animation', html='html5')

animk

